# 说明

此文档的测试采用`QA.QA_fetch_stock_list_adv`读取本地股票列表，对所有股票进行回测。回测时使用`QA.QA_fetch_stock_day_adv`读取本地存储的所有股票的日线数据进行回测。需要保证本地有相关日线数据。

本测试是对`ATR波幅通道`进行全市场测试。首先在[测试](#测试)中在全市场范围内测试`原始布林带开口`(测试时间2014-01-01~2018-12-31)。共找到`96`支股票的交易次数达到`交易次数30次以上并且R乘数>3`。

```
['000034', '000638', '000662', '000718', '000806', '000948',
       '002120', '002175', '002177', '002230', '002260', '002279',
       '002280', '002291', '002397', '002407', '002439', '002460',
       '002494', '002515', '002522', '002579', '002584', '002590',
       '002606', '002619', '002631', '002665', '002673', '002681',
       '002686', '002689', '002692', '002709', '300013', '300014',
       '300016', '300023', '300032', '300033', '300034', '300036',
       '300053', '300059', '300065', '300073', '300075', '300078',
       '300081', '300109', '300122', '300130', '300134', '300140',
       '300151', '300168', '300173', '300176', '300188', '300206',
       '300208', '300211', '300219', '300223', '300248', '300252',
       '300253', '300271', '300274', '300302', '300311', '300327',
       '300333', '300338', '300341', '300367', '300373', '300377',
       '300384', '600070', '600072', '600233', '600247', '600260',
       '600338', '600371', '600435', '600647', '600687', '600728',
       '600791', '600848', '600892', '601233', '601777', '603019']
```

按照以上筛选的股票列表，分别对三种初始资金`100,000，50,000,10,000`。进行了测试。以下列出初始资金为`10,000`时的测试回测结果：

| name                      |   盈亏总额 |   最终价值 |   交易次数 |   盈利次数 |   亏损次数 |   盈利比率 |   每笔交易平均盈亏额 |   盈利交易平均盈利额 |   亏损交易平均亏损额 |    R |   最大回撤 |   买入平均花费 |
|:--------------------------|-----------:|-----------:|-----------:|-----------:|-----------:|-----------:|---------------------:|---------------------:|---------------------:|-----:|-----------:|---------------:|
| 原始布林带开口            |    27288.7 |     138664 |        298 |        144 |        154 |   0.483221 |              53.7616 |              189.505 |             -73.1671 | 0.73 |  -0.165884 |        1270.89 |
| 原始布林带开口+1倍ATR加仓 |    58787.9 |     179856 |        541 |        305 |        236 |   0.563771 |              75.3127 |              192.747 |             -76.4567 | 0.99 |  -0.256194 |        1175.24 |

由于跟踪的股票太多，并且R值较低，所以首选选在R乘数大于3，然后按照交易次数反向排序，选择交易次数最多的10笔交易。`['002522', '300248', '300130', '002460', '300073', '300377', '002407', '600892', '002120', '300036']`。

对以上10支股票在2019年度进行了测试回测，结果如下：

| name                      |   盈亏总额 |   最终价值 |   交易次数 |   盈利次数 |   亏损次数 |   盈利比率 |   每笔交易平均盈亏额 |   盈利交易平均盈利额 |   亏损交易平均亏损额 |    R |   最大回撤 |   买入平均花费 |   策略年化收益 |
|:--------------------------|-----------:|-----------:|-----------:|-----------:|-----------:|-----------:|---------------------:|---------------------:|---------------------:|-----:|-----------:|---------------:|---------------:|
| 原始布林带开口            |    722.815 |    12728.3 |         24 |          9 |         15 |   0.375    |             -4.18383 |              80.3128 |             -54.8818 | 0.08 |  -0.169171 |       1045.4   |       0.286047 |
| 原始布林带开口+1倍ATR加仓 |   1433.84  |    11860.5 |         44 |         21 |         23 |   0.477273 |              9.75796 |              68.2781 |             -43.6735 | 0.22 |  -0.248141 |        889.201 |       0.191311 |

**从以上测试结果结合[策略测试-通道策略-布林带通道宽度.ipynb](https://github.com/GuQiangJS/gquant-example/blob/master/%E7%AD%96%E7%95%A5%E6%B5%8B%E8%AF%95-%E9%80%9A%E9%81%93%E7%AD%96%E7%95%A5-%E5%B8%83%E6%9E%97%E5%B8%A6%E9%80%9A%E9%81%93%E5%AE%BD%E5%BA%A6.ipynb)来看验证了以下观点：策略有效交易次数过少的数据并不可信。**

In [1]:
import gquant
import abupy
import QUANTAXIS as QA
import gquant.jupyter_helper
from tqdm import tqdm_notebook
from IPython.display import clear_output
import logging
gquant.jupyter_helper.init()
logging.disable(logging.DEBUG)

In [2]:
symbol_list=QA.QA_fetch_stock_list_adv()

In [3]:
# symbol_list

In [4]:
class LocalDataAPI(abupy.MarketBu.ABuDataBase.StockBaseMarket,
                   abupy.MarketBu.ABuDataBase.SupportMixin):
    def _support_market(self):
        """声明数据源支持A股"""
        return [abupy.EMarketTargetType.E_MARKET_TARGET_CN]

    def kline(self, n_folds=2, start=None, end=None):
#         import pdb
#         pdb.set_trace()
        if self._symbol.is_index():
            df=QA.QA_fetch_index_day_adv(self._symbol.symbol_code,start,end).data.copy()
        else:
            df=QA.QA_fetch_stock_day_adv(self._symbol.symbol_code,start,end).to_qfq().data.copy()
        df['pre_close'] = df['close'].shift()
        df['date'] = df.index.get_level_values(0)
        df['date'] = df['date'].dt.strftime('%Y%m%d').apply(
            lambda x: abupy.ABuDateUtil.date_str_to_int(x))
        df.index = df.index.droplevel(1)
        df.index.name = 'dt'
        df['date_week'] = df.index.dayofweek
        if start and end:
            return df[start:end]
        elif start:
            return df[start:]
        elif end:
            return df[:end]
        return df

    def minute(self, n_fold=5, *args, **kwargs):
        """分钟k线接口"""
        raise NotImplementedError('QAApi minute NotImplementedError!')

In [5]:
abupy.env.g_market_target = abupy.env.EMarketTargetType.E_MARKET_TARGET_CN
# 所有任务数据强制网络更新
abupy.env.g_data_fetch_mode = abupy.env.EMarketDataFetchMode.E_DATA_FETCH_FORCE_NET
# 使用QUATAXIS本地数据作为数据源
abupy.env.g_private_data_source = LocalDataAPI

In [6]:
from abupy.IndicatorBu.ABuNDAtr import calc_atr
from QUANTAXIS.QAIndicator.talib_series import SMA
from QUANTAXIS.QAIndicator.talib_indicators import SAR
from QUANTAXIS.QAIndicator.talib_series import BBANDS

class KLManager(abupy.AbuKLManager):
    def _fetch_pick_time_kl_pd(self, target_symbol):
        data = abupy.MarketBu.ABuSymbolPd.make_kl_df(target_symbol,start=self.benchmark.start,end=self.benchmark.end)
        if not data.empty:
            high = data['high']
            low = data['low']
            pre_close = data['pre_close']
            data['ma_slow']=SMA(data['close'],40)
            data['atr']=calc_atr(high, low, pre_close, 20)
            data['sar']=SAR(data,0.02,0.2)
            data['up']=data['ma_slow']+data['atr']
            data['low']=data['ma_slow']-data['atr']
#             data['ma_fast']=SMA(data['close'],20)
            data['atr14'] = calc_atr(high, low, pre_close, 14)
            data['atr21'] = calc_atr(high, low, pre_close, 21)
            data['bb_up'],data['bb_mid'],data['bb_low']=BBANDS(data['close'], timeperiod=20, nbdevup=1.5, nbdevdn=1.5, matype=0)
            data['WIDTH']=(data['bb_up']-data['bb_low'])/data['bb_mid']#极限宽指标
            
            data['buy']=0
            #布林带开口大小(极限宽指标)>0.2，并且收盘价在上线以上。
            data.loc[(data['WIDTH'] > 0.2)&(data['close']>data['bb_up']),'buy']=1
            data['buy']=data['buy']-data.shift()['buy']#不重复建仓
            #收盘价下穿慢速线
            data['sell']=QA.QAIndicator.CROSS(data['ma_slow'],data['close'])
            
            result = data[self.benchmark.start:self.benchmark.end]
            result.name = data.name
        else:
            return data
#         logging.info(result)
        return result

## 仓位控制方法

In [7]:
class Position1(abupy.AbuPositionBase):
    """1手"""
    def _init_self(self, **kwargs):
        """子类仓位管理针对可扩展参数的初始化"""
        pass

    def fit_position(self, factor_object):
        return 100

## 回测方法

In [8]:
def backtest(symbol,buy_factors,sell_factors,init_cash=10000,**kwargs):
    start = kwargs.pop('start','2014-01-01')
    end = kwargs.pop('end','2018-12-31')

    benchmark = abupy.AbuBenchmark('000300', start, end)
    capital = abupy.AbuCapital(init_cash, benchmark)
    klmanager=kwargs.pop('klmanager',KLManager)
    
    
    if isinstance(symbol,str):
        symbol=[symbol]
    orders_pd, action_pd, _ = abupy.ABuPickTimeExecute.do_symbols_with_same_factors(
        symbol,
        benchmark,
        buy_factors,
        sell_factors,
        capital,
        kl_pd_manager=klmanager(benchmark, capital),
        show=False)
    m = gquant.Metrics(orders_pd, action_pd, capital, benchmark)
    m.fit_metrics()
    if hasattr(m, 'act_sell') and not m.act_sell.empty:
        m.act_sell['profit_init']=m.act_sell['profit'] / m.capital.read_cash
        m.act_sell['profit_init_hunder'] = m.act_sell['profit_init'] * 100
        return m
    return None

## 回测绘图

In [9]:
def plot_cum_returns(metrics):
    metrics.benchmark_cum_returns.plot()
    metrics.algorithm_cum_returns.plot()
    gquant.plt.legend(['基准收益', '策略收益'], loc='best')
    gquant.plt.title('收益对比')
    gquant.plt.show()

def plot_win_loss_pie(metrics):
    gquant.plt.pie(x=[len(metrics.ret), len(metrics.los)],
                   labels=['盈利', '亏损'],
                   colors=['#d62728', '#2ca02c'],
                   autopct='%1.2f%%')
    gquant.plt.title('盈亏次数比')
    gquant.plt.show()
    
def plot_win_loss_bar(metrics,v='profit_cg_hunder'):
    import matplotlib.ticker
    vc=gquant.np.round(metrics.act_sell[v],0).value_counts().sort_index().to_frame()
    ax=vc.plot(kind='bar',title='盈亏百分比次数')
#     gquant.plt.axvline(x=gquant.np.where(vc.index.to_numpy()==0)[0], color='#d68a27')
#     ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%d'))
    gquant.plt.show()
    
def plot_capital_blance(metrics):
    metrics.capital.capital_pd.capital_blance.plot(title='账户价值曲线')
    metrics.capital.capital_pd.cash_blance .plot(title='账户现金曲线')
    gquant.plt.xticks(rotation=70)
    gquant.plt.legend()
    gquant.plt.show()

## 扩展回测报告

In [10]:
def get_report(m,name):
    p=m.profit_series(name='回测-{}'.format(name))
    p['平均获利期望'] = m.gains_mean  #平均获利期望\n",
    p['平均亏损期望'] = m.losses_mean  #平均亏损期望\n",
    p['盈亏比'] = m.win_loss_profit_rate  #盈亏比\n",
    p['策略年化收益'] = m.algorithm_annualized_returns  #策略年化收益\n",
    p['基准年化收益'] = m.benchmark_annualized_returns  #基准年化收益\n",
    c = m.order_has_ret
    p['赢利交易平均持股天数'] = c[c.profit > 0].keep_days.values.mean()  #赢利交易平均持股天数\n",
    p['亏损交易平均持股天数'] = c[c.profit <= 0].keep_days.values.mean()  #亏损交易平均持股天数\n",
    p = gquant.pd.to_numeric(p)
    return p.to_frame().round(2).T

## 买卖策略

In [11]:
class BuyStrategy1(gquant.abupy.AbuFactorBuyTD, gquant.abupy.BuyCallMixin):
    def _init_self(self, **kwargs):
        self.factor_name = self.__class__.__name__

    def fit_day(self, today):
        if today['buy']:
            return self.buy_tomorrow()
        return None
    
class SellStrategy1(gquant.abupy.AbuFactorSellBase):
    def _init_self(self, **kwargs):
        self.factor_name = self.__class__.__name__

    def support_direction(self):
        """支持的方向，只支持正向"""
        return [abupy.ESupportDirection.DIRECTION_CAll.value]

    def fit_day(self, today, orders):
        if today['sell']:
            for order in orders:
                self.sell_tomorrow(order)

### 买-当日价格超过购买价+1倍购买日ATR

In [13]:
class BuyStrategy2(gquant.abupy.AbuFactorBuyTD, gquant.abupy.BuyCallMixin):
    def _init_self(self, **kwargs):
        self.factor_name = self.__class__.__name__

    def fit_day(self, today):
        if self.orders:
            o=self.orders[-1]
            if o.sell_date is None:
                buy=self.kl_pd[self.kl_pd['date']==o.buy_date]#最后一次购买当日的信息
                atr=buy.iloc[0]['atr']#最后一次购买当日的atr数据
                if today.close>today.up and today.close>o.buy_price+atr:
                    #当天价格在波幅通道之上，并且当天收盘价大于最后一次购买价格+1倍购买日atr（可加仓价）
#                     import logging
#                     logging.info('{}:当日收盘价:{:.2f},当日波幅上轨:{:.2f},购买日:{},购买价格:{:.2f},购买日ATR:{:.2f}'.format(today.date,today.close,today.up,o.buy_date,o.buy_price,atr))
                    return self.buy_tomorrow()

# 训练测试

In [14]:
ts = {
    '原始布林带开口': {
        # 买入因子
        'buy_factors': [
            {
                'class': BuyStrategy1,
                # 仓位管理
                'position': {
                    'class': Position1
                }
            },
        ],
        # 卖出因子
        'sell_factors': [{
            'class': SellStrategy1,
        }]
    },
    '原始布林带开口+1倍ATR加仓': {
        # 买入因子
        'buy_factors': [
            {
                'class': BuyStrategy1,
                # 仓位管理
                'position': {
                    'class': Position1
                }
            },
            {
                'class': BuyStrategy2,
                # 仓位管理
                'position': {
                    'class': Position1
                }
            }
        ],
        # 卖出因子
        'sell_factors': [{
            'class': SellStrategy1,
        }]
    },
}

## 全市场

使用原始波幅测试所有股票。测试期间为`2014-01-01~2018-12-31`。

结果存储在`datas布林带通道宽度-全市场测试-2014至2018.xlsx`文件中。如果文件存在就读取，不再重复测试（测试时间过长）。

**由于测试环境问题，当时未能保存为xlsx格式（虽然文件后缀名还是xlsx。因为github上做了csv文件后缀名被排除上传的设置）。但是csv文件读取后的效果是否需要做二次转换，并未测试。**

In [ ]:
def getreport(bts):
    report=[]
    for symbol,metrics in bts.items():
        for name,m in metrics.items():
            if m is None:
                continue
            p=m.profit_series(name=name)
            p=gquant.pd.to_numeric(p)
            p['code']=symbol
            report.append(p)
    if not report:
        return gquant.pd.DataFrame({'R':[]})
    report=gquant.pd.concat(report, axis=1).T
    for col in report.columns:
        if col=='code':
            continue
        report[col]=gquant.pd.to_numeric(report[col])
    return report

import os
import pickle
data_file='datas\布林带通道宽度-全市场测试-2014至2018.xlsx'
if os.path.exists(data_file):
    report=gquant.pd.read_excel('datas\布林带通道宽度-全市场测试-2014至2018.xlsx',encoding='utf_8_sig',dtype={'code':str},index_col=0)
else:
    bts={}

    for symbol in tqdm_notebook(symbol_list.code):
        bts[symbol]={}
        for k,v in ts.items():
            if k!='原始布林带开口':
                continue
            try:
                m=backtest(symbol,v['buy_factors'],v['sell_factors'],start='2014-01-01',end='2018-12-31')
                if m is None:
                    continue
                bts[symbol][k]=m
            except:
                pass

    report=getreport(bts)
    report.to_excel(data_file,encoding='utf_8_sig')
    
#     with open('datas\ATR波幅通道-全市场测试-Metrics-2014至2018.pickle', 'wb') as f:
#         pickle.dump(bts, f, pickle.HIGHEST_PROTOCOL)

clear_output(wait=True)

In [ ]:
# with open('datas\ATR波幅通道-全市场测试-Metrics-2014至2018.pickle', 'rb') as f:
#     data = pickle.load(f)

### 结果

In [16]:
report.loc[(report['交易次数']>30)&(report['R']>3)].sort_values('R',ascending=False)

,盈亏总额,最终价值,交易次数,盈利次数,亏损次数,盈利比率,每笔交易平均盈亏额,盈利交易平均盈利额,亏损交易平均亏损额,R,最大回撤,买入平均花费,code
原始布林带开口,24384.0383,34156.933,48.0,42.0,6.0,0.8750,503.1149,580.5723,-39.0869,12.87,-0.3551,1190.7220,002522
原始布林带开口,30337.8064,40028.376,40.0,35.0,5.0,0.8750,750.4810,866.7945,-63.7136,11.78,-0.2946,739.7111,300059
原始布林带开口,13100.7183,22964.098,33.0,29.0,4.0,0.8788,392.7302,451.7489,-35.1556,11.17,-0.2783,1319.3083,300327
原始布林带开口,17671.9693,27260.239,38.0,28.0,10.0,0.7368,454.0772,631.1418,-41.7035,10.89,-0.3003,1099.8975,000034
原始布林带开口,18239.5162,27998.931,42.0,36.0,6.0,0.8571,428.4144,506.6532,-41.0187,10.44,-0.2458,1100.1854,000662
...,...,...,...,...,...,...,...,...,...,...,...,...,...
原始布林带开口,8642.7166,17647.556,37.0,22.0,15.0,0.5946,206.6253,392.8508,-66.5054,3.11,-0.2189,1015.9591,600371
原始布林带开口,11191.6899,19782.034,40.0,22.0,18.0,0.5500,244.4744,508.7132,-78.4842,3.11,-0.3801,986.4681,300173
原始布林带开口,8357.5677,17591.651,35.0,24.0,11.0,0.6857,216.8360,348.2320,-69.8461,3.10,-0.2477,912.9957,300219
原始布林带开口,8085.7540,17168.947,33.0,20.0,13.0,0.6061,217.1723,404.2877,-70.6976,3.07,-0.2293,1333.1519,600435


In [17]:
report.loc[(report['R']>3)].sort_values('交易次数',ascending=False).head(10).sort_values('R',ascending=False)

,盈亏总额,最终价值,交易次数,盈利次数,亏损次数,盈利比率,每笔交易平均盈亏额,盈利交易平均盈利额,亏损交易平均亏损额,R,最大回撤,买入平均花费,code
原始布林带开口,24384.0383,34156.933,48.0,42.0,6.0,0.8750,503.1149,580.5723,-39.0869,12.87,-0.3551,1190.7220,002522
原始布林带开口,34059.4971,42661.600,47.0,30.0,17.0,0.6383,689.6462,1135.3166,-96.8308,7.12,-0.3527,1319.0011,300248
原始布林带开口,24278.3656,32926.462,51.0,34.0,17.0,0.6667,449.4003,714.0696,-79.9382,5.62,-0.3268,1275.7331,300130
原始布林带开口,28471.0144,37155.049,64.0,48.0,16.0,0.7500,424.1672,593.1461,-82.7698,5.12,-0.2139,1582.7907,002460
原始布林带开口,20075.4960,29149.518,49.0,35.0,14.0,0.7143,385.4290,573.5856,-84.9624,4.54,-0.2413,1744.8770,300073
原始布林带开口,15994.4884,24969.495,67.0,47.0,20.0,0.7015,223.3551,340.3083,-51.4849,4.34,-0.3334,1048.7904,300377
原始布林带开口,17783.7077,26904.184,52.0,41.0,11.0,0.7885,324.9797,433.7490,-80.4329,4.04,-0.2818,1550.7450,002407
原始布林带开口,12817.6999,21756.837,47.0,30.0,17.0,0.6383,250.0671,427.2567,-62.6203,3.99,-0.2285,1087.1638,600892
原始布林带开口,7419.4861,17101.816,45.0,38.0,7.0,0.8444,157.7678,195.2496,-45.7047,3.45,-0.3688,965.4481,002120
原始布林带开口,12927.7047,21445.816,48.0,28.0,20.0,0.5833,238.3799,461.7037,-74.2735,3.21,-0.4180,1284.2796,300036


# 2014~2018间符合测试方法的列表

- 成交数量大于30 & R乘数大于3。
- 因为跟踪的股票数量过多，所以首选选在R乘数大于3，然后按照交易次数反向排序，选择交易次数最多的10笔交易。

In [18]:
symbols=report.loc[(report['交易次数']>30)&(report['R']>3)].code.values #成交数量大于30 & R乘数大于3
symbols_10=report.loc[(report['R']>3)].sort_values('交易次数',ascending=False).head(10).sort_values('R',ascending=False).code.values #选择交易次数最多的10笔交易

In [32]:
symbols

array(['000034', '000638', '000662', '000718', '000806', '000948',
       '002120', '002175', '002177', '002230', '002260', '002279',
       '002280', '002291', '002397', '002407', '002439', '002460',
       '002494', '002515', '002522', '002579', '002584', '002590',
       '002606', '002619', '002631', '002665', '002673', '002681',
       '002686', '002689', '002692', '002709', '300013', '300014',
       '300016', '300023', '300032', '300033', '300034', '300036',
       '300053', '300059', '300065', '300073', '300075', '300078',
       '300081', '300109', '300122', '300130', '300134', '300140',
       '300151', '300168', '300173', '300176', '300188', '300206',
       '300208', '300211', '300219', '300223', '300248', '300252',
       '300253', '300271', '300274', '300302', '300311', '300327',
       '300333', '300338', '300341', '300367', '300373', '300377',
       '300384', '600070', '600072', '600233', '600247', '600260',
       '600338', '600371', '600435', '600647', '600687', '6007

In [19]:
symbols_10

array(['002522', '300248', '300130', '002460', '300073', '300377',
       '002407', '600892', '002120', '300036'], dtype=object)

# 2019年度测试回测

## 全市场

2019年度全市场所有股票采用ATR波幅规则进行回测（因为股票数量过多，所以将初始资金由`10,000`提高到`100,000`）。盈利比率大于`50%`。但R值较低。最大回撤在`13%`左右。

| name                      |   盈亏总额 |   最终价值 |   交易次数 |   盈利次数 |   亏损次数 |   盈利比率 |   每笔交易平均盈亏额 |   盈利交易平均盈利额 |   亏损交易平均亏损额 |    R |   最大回撤 |   买入平均花费 |
|:--------------------------|-----------:|-----------:|-----------:|-----------:|-----------:|-----------:|---------------------:|---------------------:|---------------------:|-----:|-----------:|---------------:|
| 原始布林带开口            |    97614.1 |     153700 |        928 |        499 |        429 |   0.537716 |              79.4076 |              195.619 |             -55.7666 | 1.42 |  -0.137009 |        892.864 |
| 原始布林带开口+1倍ATR加仓 |   115150   |     152669 |        998 |        609 |        389 |   0.61022  |              97.3582 |              189.081 |             -46.2382 | 2.11 |  -0.132333 |        861.134 |

In [ ]:
bt_2019_all = {}
for n, t in tqdm_notebook(ts.items()):
    mc = backtest(list(symbol_list.code),
                  t['buy_factors'],
                  t['sell_factors'],
                  start='2019-01-01',
                  end='2019-12-31',
                  init_cash=100000)
    if mc:
        d = mc.profit_series().T
        d['name'] = n
        bt_2019_all[n] = d
clear_output(wait=True)

In [21]:
gquant.pd.set_option('display.float_format',lambda x : '%.2f' % x)
print(gquant.pd.concat([pd.to_frame().T for pd in bt_2019_all.values()]).set_index('name').to_markdown())

| name                      |   盈亏总额 |   最终价值 |   交易次数 |   盈利次数 |   亏损次数 |   盈利比率 |   每笔交易平均盈亏额 |   盈利交易平均盈利额 |   亏损交易平均亏损额 |    R |   最大回撤 |   买入平均花费 |
|:--------------------------|-----------:|-----------:|-----------:|-----------:|-----------:|-----------:|---------------------:|---------------------:|---------------------:|-----:|-----------:|---------------:|
| 原始布林带开口            |    97614.1 |     153700 |        928 |        499 |        429 |   0.537716 |              79.4076 |              195.619 |             -55.7666 | 1.42 |  -0.137009 |        892.864 |
| 原始布林带开口+1倍ATR加仓 |   115150   |     152669 |        998 |        609 |        389 |   0.61022  |              97.3582 |              189.081 |             -46.2382 | 2.11 |  -0.132333 |        861.134 |


## 筛选列表回测

根据[训练回测](#训练回测)中从全市场（2014~2018年度间所有股票的筛选结果中）选出的96支股票进行测试

### 初始资金100,000

由于股票还是较多，所以依然采用的是初始资金`100,000`元。

结果：相对于[完全市场结果](#%E5%85%A8%E5%B8%82%E5%9C%BA)来看，R值均降低较大。最大回撤也有增加。

| name                      |   盈亏总额 |   最终价值 |   交易次数 |   盈利次数 |   亏损次数 |   盈利比率 |   每笔交易平均盈亏额 |   盈利交易平均盈利额 |   亏损交易平均亏损额 |    R |   最大回撤 |   买入平均花费 |
|:--------------------------|-----------:|-----------:|-----------:|-----------:|-----------:|-----------:|---------------------:|---------------------:|---------------------:|-----:|-----------:|---------------:|
| 原始布林带开口            |    27288.7 |     138664 |        298 |        144 |        154 |   0.483221 |              53.7616 |              189.505 |             -73.1671 | 0.73 |  -0.165884 |        1270.89 |
| 原始布林带开口+1倍ATR加仓 |    58787.9 |     179856 |        541 |        305 |        236 |   0.563771 |              75.3127 |              192.747 |             -76.4567 | 0.99 |  -0.256194 |        1175.24 |

In [ ]:
bt_2019_symbols = {}
for n, t in tqdm_notebook(ts.items()):
    mc = backtest(symbols,
                  t['buy_factors'],
                  t['sell_factors'],
                  start='2019-01-01',
                  end='2019-12-31',
                  init_cash=100000)
    if mc:
        d = mc.profit_series().T
        d['name'] = n
        bt_2019_symbols[n] = d
clear_output(wait=True)

In [23]:
gquant.pd.set_option('display.float_format',lambda x : '%.2f' % x)
print(gquant.pd.concat([pd.to_frame().T for pd in bt_2019_symbols.values()]).set_index('name').to_markdown())

| name                      |   盈亏总额 |   最终价值 |   交易次数 |   盈利次数 |   亏损次数 |   盈利比率 |   每笔交易平均盈亏额 |   盈利交易平均盈利额 |   亏损交易平均亏损额 |    R |   最大回撤 |   买入平均花费 |
|:--------------------------|-----------:|-----------:|-----------:|-----------:|-----------:|-----------:|---------------------:|---------------------:|---------------------:|-----:|-----------:|---------------:|
| 原始布林带开口            |    27288.7 |     138664 |        298 |        144 |        154 |   0.483221 |              53.7616 |              189.505 |             -73.1671 | 0.73 |  -0.165884 |        1270.89 |
| 原始布林带开口+1倍ATR加仓 |    58787.9 |     179856 |        541 |        305 |        236 |   0.563771 |              75.3127 |              192.747 |             -76.4567 | 0.99 |  -0.256194 |        1175.24 |


### 初始资金50,,000

本次测试依然沿用筛选后的195支股票进行测试，但是初始资金改为`50,000`元。

| name                      |   盈亏总额 |   最终价值 |   交易次数 |   盈利次数 |   亏损次数 |   盈利比率 |   每笔交易平均盈亏额 |   盈利交易平均盈利额 |   亏损交易平均亏损额 |    R |   最大回撤 |   买入平均花费 |
|:--------------------------|-----------:|-----------:|-----------:|-----------:|-----------:|-----------:|---------------------:|---------------------:|---------------------:|-----:|-----------:|---------------:|
| 原始布林带开口            |    22808.4 |    81489.9 |        204 |         97 |        107 |    0.47549 |              73.9976 |              235.138 |             -72.0828 | 1.03 |  -0.22584  |        1228.38 |
| 原始布林带开口+1倍ATR加仓 |    32661.3 |    86284.2 |        356 |        188 |        168 |    0.52809 |              50.7901 |              173.73  |             -86.7859 | 0.59 |  -0.333339 |        1032.81 |

In [ ]:
bt_2019_symbols_5 = {}
for n, t in tqdm_notebook(ts.items()):
    mc = backtest(symbols,
                  t['buy_factors'],
                  t['sell_factors'],
                  start='2019-01-01',
                  end='2019-12-31',
                  init_cash=50000)
    if mc:
        d = mc.profit_series().T
        d['name'] = n
        bt_2019_symbols_5[n] = d
clear_output(wait=True)

In [25]:
gquant.pd.set_option('display.float_format',lambda x : '%.2f' % x)
print(gquant.pd.concat([pd.to_frame().T for pd in bt_2019_symbols_5.values()]).set_index('name').to_markdown())

| name                      |   盈亏总额 |   最终价值 |   交易次数 |   盈利次数 |   亏损次数 |   盈利比率 |   每笔交易平均盈亏额 |   盈利交易平均盈利额 |   亏损交易平均亏损额 |    R |   最大回撤 |   买入平均花费 |
|:--------------------------|-----------:|-----------:|-----------:|-----------:|-----------:|-----------:|---------------------:|---------------------:|---------------------:|-----:|-----------:|---------------:|
| 原始布林带开口            |    22808.4 |    81489.9 |        204 |         97 |        107 |    0.47549 |              73.9976 |              235.138 |             -72.0828 | 1.03 |  -0.22584  |        1228.38 |
| 原始布林带开口+1倍ATR加仓 |    32661.3 |    86284.2 |        356 |        188 |        168 |    0.52809 |              50.7901 |              173.73  |             -86.7859 | 0.59 |  -0.333339 |        1032.81 |


### 初始资金10,000

本次测试依然沿用筛选后的195支股票进行测试，但是初始资金改为`10,000`元。

| name                      |   盈亏总额 |   最终价值 |   交易次数 |   盈利次数 |   亏损次数 |   盈利比率 |   每笔交易平均盈亏额 |   盈利交易平均盈利额 |   亏损交易平均亏损额 |    R |   最大回撤 |   买入平均花费 |
|:--------------------------|-----------:|-----------:|-----------:|-----------:|-----------:|-----------:|---------------------:|---------------------:|---------------------:|-----:|-----------:|---------------:|
| 原始布林带开口            |    4742    |    13109.2 |         84 |         35 |         49 |   0.416667 |              13.8748 |              135.486 |             -72.99   | 0.19 |  -0.309914 |        909.033 |
| 原始布林带开口+1倍ATR加仓 |    7821.26 |    15128.1 |        102 |         64 |         38 |   0.627451 |              52.8502 |              122.207 |             -63.9617 | 0.83 |  -0.348772 |        792.014 |

In [ ]:
bt_2019_symbols_1 = {}
for n, t in tqdm_notebook(ts.items()):
    mc = backtest(symbols,
                  t['buy_factors'],
                  t['sell_factors'],
                  start='2019-01-01',
                  end='2019-12-31')
    if mc:
        d = mc.profit_series().T
        d['name'] = n
        bt_2019_symbols_1[n] = d
clear_output(wait=True)

In [27]:
gquant.pd.set_option('display.float_format',lambda x : '%.2f' % x)
print(gquant.pd.concat([pd.to_frame().T for pd in bt_2019_symbols_1.values()]).set_index('name').to_markdown())

| name                      |   盈亏总额 |   最终价值 |   交易次数 |   盈利次数 |   亏损次数 |   盈利比率 |   每笔交易平均盈亏额 |   盈利交易平均盈利额 |   亏损交易平均亏损额 |    R |   最大回撤 |   买入平均花费 |
|:--------------------------|-----------:|-----------:|-----------:|-----------:|-----------:|-----------:|---------------------:|---------------------:|---------------------:|-----:|-----------:|---------------:|
| 原始布林带开口            |    4742    |    13109.2 |         84 |         35 |         49 |   0.416667 |              13.8748 |              135.486 |             -72.99   | 0.19 |  -0.309914 |        909.033 |
| 原始布林带开口+1倍ATR加仓 |    7821.26 |    15128.1 |        102 |         64 |         38 |   0.627451 |              52.8502 |              122.207 |             -63.9617 | 0.83 |  -0.348772 |        792.014 |


## 跟踪10支股票回测

| name                      |   盈亏总额 |   最终价值 |   交易次数 |   盈利次数 |   亏损次数 |   盈利比率 |   每笔交易平均盈亏额 |   盈利交易平均盈利额 |   亏损交易平均亏损额 |    R |   最大回撤 |   买入平均花费 |   策略年化收益 |
|:--------------------------|-----------:|-----------:|-----------:|-----------:|-----------:|-----------:|---------------------:|---------------------:|---------------------:|-----:|-----------:|---------------:|---------------:|
| 原始布林带开口            |    722.815 |    12728.3 |         24 |          9 |         15 |   0.375    |             -4.18383 |              80.3128 |             -54.8818 | 0.08 |  -0.169171 |       1045.4   |       0.286047 |
| 原始布林带开口+1倍ATR加仓 |   1433.84  |    11860.5 |         44 |         21 |         23 |   0.477273 |              9.75796 |              68.2781 |             -43.6735 | 0.22 |  -0.248141 |        889.201 |       0.191311 |

In [28]:
bt_2019_symbols_1_1 = {}
bt_2019_symbols_1_1_metics={}
for n, t in tqdm_notebook(ts.items()):
    mc = backtest(symbols_10,
                  t['buy_factors'],
                  t['sell_factors'],
                  start='2019-01-01',
                  end='2019-12-31')
    if mc:
        d = mc.profit_series().T
        d['name'] = n
        d['策略年化收益']=mc.algorithm_annualized_returns#策略年化收益
        bt_2019_symbols_1_1_metics[n]=mc
        bt_2019_symbols_1_1[n] = d
clear_output(wait=True)

Box(children=(Text(value='pid=1504 begin work'), FloatProgress(value=0.0)))

capital.apply_action::1.23%

FloatProgress(value=0.0)

Box(children=(Text(value='pid=1504 begin work'), FloatProgress(value=0.0)))

capital.apply_action::0.24%

FloatProgress(value=0.0)

capital.apply_action::100.0%


In [29]:
gquant.pd.set_option('display.float_format',lambda x : '%.2f' % x)
print(gquant.pd.concat([pd.to_frame().T for pd in bt_2019_symbols_1_1.values()]).set_index('name').round(2).to_markdown())

| name                      |   盈亏总额 |   最终价值 |   交易次数 |   盈利次数 |   亏损次数 |   盈利比率 |   每笔交易平均盈亏额 |   盈利交易平均盈利额 |   亏损交易平均亏损额 |    R |   最大回撤 |   买入平均花费 |   策略年化收益 |
|:--------------------------|-----------:|-----------:|-----------:|-----------:|-----------:|-----------:|---------------------:|---------------------:|---------------------:|-----:|-----------:|---------------:|---------------:|
| 原始布林带开口            |    722.815 |    12728.3 |         24 |          9 |         15 |   0.375    |             -4.18383 |              80.3128 |             -54.8818 | 0.08 |  -0.169171 |       1045.4   |       0.286047 |
| 原始布林带开口+1倍ATR加仓 |   1433.84  |    11860.5 |         44 |         21 |         23 |   0.477273 |              9.75796 |              68.2781 |             -43.6735 | 0.22 |  -0.248141 |        889.201 |       0.191311 |
